In [ ]:
import pandas as pd
import numpy as np

Chargement du fichier labellisé

In [ ]:
df = pd.read_csv("results-corrected.csv", dtype=str)

On gère deux listes : 
- la liste des types détectés par CSV Detective
- la liste des types qu'on a utilisé pour la labellisation

In [ ]:
csvdetective_types = df.column_type.unique()

In [ ]:
label_types_combinaisons = df.value.unique()

In [ ]:
label_types = []
for label_type in label_types_combinaisons:
    for val in eval(label_type):
        if val not in label_types:
            label_types.append(val)

In [ ]:
df.head()

La liste des résultats est incomplète car il n'y a qu'un type "column_type" indiqué dans celui-ci. Or il pourrait y avoir potentiellement plus de types détectés pour une même colonne. 
Pour cela, on va chercher le fichier full.csv et on ajoutera dans le fichier les autres types détectés sur les colonnes du fichier result.csv.
On aura donc une liste exhaustive

In [ ]:
full = pd.read_csv("full.csv",dtype=str, error_bad_lines=False,engine='python')

In [ ]:
new = []
for index,row in df.iterrows():
    ct = row['column_type']
    new.append(row.to_dict())
    nbrows = full[(full['resource_id'] == row.resource_id) & (full['header'] == row.header) & (full['resource_url'] == row.resource_url)]
    if(nbrows.shape[0] > 1):
        for index2,row2 in nbrows.iterrows():
            if(ct != row2['column_type']):
                rowbis = {}
                rowbis = row
                rowbis['column_type'] = row2['column_type']
                rowbis['index'] = row2['Unnamed: 0']
                rowbis['score_ml'] = row2['score_ml']
                rowbis['score_rb'] = row2['score_rb']
                new.append(rowbis.to_dict())

On avait initialement dans le fichier result.csv :

In [9]:
df.shape

(5534, 16)

In [10]:
df.drop_duplicates().shape

(5325, 16)

Dans le fichier remodelé on a : 

In [11]:
df = pd.DataFrame(new)

In [12]:
df.shape

(17201, 16)

In [13]:
df = df.drop_duplicates(keep="first")

In [14]:
df.shape

(14456, 16)

Cast colonnes scores

In [15]:
df.score_ml = df.score_ml.astype(float)
df.score_rb = df.score_rb.astype(float)

In [16]:
def ifFound(val, arr):
    if(val in eval(arr)):
        return True
    else:
        return False

In [17]:
def analysis(dfi):
    # On vérifie si le type trouvé par CSV Detective est bien présent dans la labellisation
    dfi['isFoundByCsvDetective'] = dfi.apply(lambda x: ifFound(x.column_type, x.value), axis=1)
    results = dfi[['column_type', 'isFoundByCsvDetective','index']].groupby(['column_type','isFoundByCsvDetective'],as_index=False).count()
    # Récupération du nombre total de détection par CSV Detective pour chacun des types
    total = pd.DataFrame(dfi.column_type.value_counts())
    total.reset_index(level=0, inplace=True)
    total = total.rename(columns={'index':'column_type','column_type':'total'})
    # Merge des deux dataframe pour pouvoir calculer indicateurs
    results = pd.merge(results,total,on='column_type',how='left')
    results = results[results['isFoundByCsvDetective'] == True][['column_type','index','total']]
    
    # Récupération du nombre de détection juste par CSV Detective pour chacun des types
    arr = {}
    for ct in csvdetective_types:
        arr[ct] = 0
    for index, row in dfi.iterrows():
        for ct in csvdetective_types:
            if(ct in eval(row['value'])):
                arr[ct] = arr[ct] + 1
    labeldf = pd.DataFrame.from_dict(arr, orient='index')
    labeldf.reset_index(level=0, inplace=True)
    labeldf = labeldf.rename(columns={'index':'column_type',0:'total_label'})
    
    # Merge des deux dataframe pour pouvoir calculer indicateurs
    results = pd.merge(results,labeldf,on='column_type',how='left')
    res = results[['column_type','index','total','total_label']]
    
    # Calcul des indicateurs VP VN FP FN Recall et Precision
    res = res.rename(columns={'index':'vrai_positif'})
    res['faux_positif'] = res['total_label'] - res['vrai_positif']
    res['faux_negatif'] = res['total'] - res['vrai_positif']
    res['vrai_negatif'] = (dfi.shape[0] - res['total_label']) - res['faux_negatif']
    res = res[['column_type','vrai_positif','faux_positif','vrai_negatif','faux_negatif']]
    res['precision'] = res['vrai_positif'] / (res['vrai_positif'] + res['faux_positif'])
    res['rappel'] = res['vrai_positif'] / (res['vrai_positif'] + res['faux_negatif'])
    
    return res

Analyse basé sur un seuil. 

Celui-ci correspond au score renvoyé par CSV Detective.

Un seuil de 0.5 semble pertinent pour indiquer que csv-detective est plutôt sûr de son choix.

L'analyse peut être appliqué sur le score RB ou ML

In [18]:
SEUIL = 0.5

Fonction qui permet de :
- garder les lignes dont le score est inférieur au seuil et dont le type n'est pas détecté
- enlever les lignes dont le score est inférieur au seuil et dont au moins un type est détecté sur la colonne concernée

In [20]:
def remove_duplicates_none(dfi):
    new = []
    for index,row in dfi.iterrows():
        if(dfi[(dfi['resource_id'] == row.resource_id) & (dfi['header'] == row.header) & (dfi['resource_url'] == row.resource_url)].shape[0] > 1):
            if(row['column_type'] is not None):
                new.append(row.to_dict())
        else:
            new.append(row.to_dict())
    dfi = pd.DataFrame(new)
    return dfi


In [21]:
res_rb = df.copy()
res_rb['column_type'] = np.where(res_rb['score_rb'] < SEUIL, None, res_rb['column_type'])
res_rb = remove_duplicates_none(res_rb)

In [22]:
analysis(res_rb)

,column_type,vrai_positif,faux_positif,vrai_negatif,faux_negatif,precision,rappel
0,adresse,59,43,7752,29,0.578431,0.670455
1,booleen,158,33,7196,496,0.827225,0.241590
2,code_commune_insee,64,8,7758,53,0.888889,0.547009
3,code_departement,130,3,7428,322,0.977444,0.287611
4,code_postal,95,73,7714,1,0.565476,0.989583
5,code_region,63,66,7732,22,0.488372,0.741176
6,commune,98,39,7622,124,0.715328,0.441441
7,date,239,54,7565,25,0.815700,0.905303
8,date_fr,1,0,7882,0,1.000000,1.000000
9,departement,49,7,7702,125,0.875000,0.281609


In [23]:
res_rb[res_rb['value'].str.contains('longitude_wgs')][['resource_id','header','column_type','shuffle_serie','value']]

,resource_id,header,column_type,shuffle_serie,value
980,5261b353-f751-490d-83a0-10cf1c2a0ea7,coordinates.longitude,longitude_wgs,"[""1.249727"", ""6.360289"", ""-1.717238"", ""3.87471...",['longitude_wgs']
981,5261b353-f751-490d-83a0-10cf1c2a0ea7,coordinates.longitude,latitude_wgs,"[""1.249727"", ""6.360289"", ""-1.717238"", ""3.87471...",['longitude_wgs']
982,5261b353-f751-490d-83a0-10cf1c2a0ea7,coordinates.longitude,longitude_l93,"[""1.249727"", ""6.360289"", ""-1.717238"", ""3.87471...",['longitude_wgs']
983,5261b353-f751-490d-83a0-10cf1c2a0ea7,coordinates.longitude,latitude_wgs_fr_metropole,"[""1.249727"", ""6.360289"", ""-1.717238"", ""3.87471...",['longitude_wgs']
984,482560c9-2a97-4110-aba7-24ea99ad1f34,coordinates.longitude,longitude_l93,"[""0.36694"", ""2.22971979999999"", ""2.057573"", ""2...",['longitude_wgs']
...,...,...,...,...,...
7728,50625621-18bd-43cb-8fde-6b8c24bdabb3,Xlongitude,latitude_wgs_fr_metropole,"[""-1.376317"", ""1.814654"", ""4.29988300"", ""7.109...",['longitude_wgs']
7729,d50363f3-bb74-4f81-ab16-fccb29af7973,Xlongitude,latitude_wgs,"[""2.20703"", ""3.91491"", ""4.12818100"", ""0.08471""...",['longitude_wgs']
7730,d50363f3-bb74-4f81-ab16-fccb29af7973,Xlongitude,longitude_wgs,"[""2.20703"", ""3.91491"", ""4.12818100"", ""0.08471""...",['longitude_wgs']
7731,d50363f3-bb74-4f81-ab16-fccb29af7973,Xlongitude,latitude_wgs_fr_metropole,"[""2.20703"", ""3.91491"", ""4.12818100"", ""0.08471""...",['longitude_wgs']


In [24]:
full[(full['resource_id'] == 'dbc511e2-2b24-4973-a858-f09408c5b63a') & (full['header'] == 'compte_instagram')]

,Unnamed: 0,column_type,score_ml,score_rb,header,resource_id,resource_url,sep,encoding,resource_title,dataset_title,headers,minio_url,shuffle_serie
63715,63891,commune,0.27699999999999997,0.0,compte_instagram,dbc511e2-2b24-4973-a858-f09408c5b63a,https://data.enseignementsup-recherche.gouv.fr...,;,utf-8,Fichier CSV,Institutions partenaires des pôles étudiants p...,"['date_de_creation', 'identifiant', 'libelle_c...",http://localhost:9000/link-proxy-files/data.en...,"[""https://www.instagram.com/universite_paris_s..."


In [25]:
res_ml = df.copy()
res_ml['column_type'] = np.where(res_ml['score_ml'] < SEUIL, None, res_ml['column_type'])
res_ml = remove_duplicates_none(res_ml)

In [26]:
analysis(res_ml)

,column_type,vrai_positif,faux_positif,vrai_negatif,faux_negatif,precision,rappel
0,booleen,75,71,2713,137,0.513699,0.353774
1,code_commune_insee,2,23,2905,66,0.080000,0.029412
2,code_departement,78,2,2882,34,0.975000,0.696429
3,code_postal,56,38,2891,11,0.595745,0.835821
4,code_region,60,0,2889,47,1.000000,0.560748
5,commune,99,30,2654,213,0.767442,0.317308
6,date,251,34,2659,52,0.880702,0.828383
7,longitude_l93,6,4,2977,9,0.600000,0.400000
8,sexe,13,10,2973,0,0.565217,1.000000
9,siret,6,1,2809,180,0.857143,0.032258


In [30]:
res_rb.to_csv("res_rb.csv",index=False)

In [31]:
res_ml.to_csv("res_ml.csv",index=False)

In [32]:
analysis_rb = analysis(res_rb)

In [33]:
analysis_rb.to_csv("analysis_rb.csv",index=False)

In [34]:
analysis_ml = analysis(res_ml)

In [35]:
analysis_ml.to_csv("analysis_ml.csv",index=False)

In [51]:
analysis_rb[analysis_rb['column_type'] == 'adresse']['vrai_positif']

0    59
Name: vrai_positif, dtype: int64

In [39]:
list(res_rb.column_type.unique())

['booleen',
 'longitude_l93',
 'latitude_wgs',
 'latitude_wgs_fr_metropole',
 'longitude_wgs',
 'code_postal',
 'code_commune_insee',
 'sexe',
 'tel_fr',
 None,
 'adresse',
 'code_departement',
 'commune',
 'iso_country_code',
 'year',
 'departement',
 'csp_insee',
 'insee_ape700',
 'insee_canton',
 'region',
 'url',
 'code_fantoir',
 'siren',
 'code_region',
 'date',
 'longitude_wgs_fr_metropole',
 'latlon_wgs',
 'datetime_iso',
 'code_waldec',
 'email',
 'uai',
 'json_geojson',
 'jour_de_la_semaine',
 'siret',
 'pays',
 'date_fr',
 'latitude_l93']

In [42]:
toto = list(res_rb.column_type.unique())

In [44]:
toto.remove(None)

In [46]:
list(res_rb.column_type.unique()).remove(None)

In [47]:
toto

['booleen',
 'longitude_l93',
 'latitude_wgs',
 'latitude_wgs_fr_metropole',
 'longitude_wgs',
 'code_postal',
 'code_commune_insee',
 'sexe',
 'tel_fr',
 'adresse',
 'code_departement',
 'commune',
 'iso_country_code',
 'year',
 'departement',
 'csp_insee',
 'insee_ape700',
 'insee_canton',
 'region',
 'url',
 'code_fantoir',
 'siren',
 'code_region',
 'date',
 'longitude_wgs_fr_metropole',
 'latlon_wgs',
 'datetime_iso',
 'code_waldec',
 'email',
 'uai',
 'json_geojson',
 'jour_de_la_semaine',
 'siret',
 'pays',
 'date_fr',
 'latitude_l93']

In [66]:
toto = res_rb[res_rb['column_type'] == 'adresse'][['header','shuffle_serie']].sample(frac=1)[:1]

In [67]:
toto.header.iloc[0]

'location_giveBlood'

In [77]:
import json

In [79]:
pd.DataFrame(json.loads(toto.shuffle_serie.iloc[0]))

,0
0,True
1,True
2,True
3,True
4,True
...,...
95,True
96,True
97,True
98,True


In [82]:
out = res_rb[res_rb['column_type'] == 'adresse'][['header','shuffle_serie']].sample(frac=1)[:1]

In [83]:
out

,header,shuffle_serie
191,adresse_geographique,"[""47 Rue Pasteur 77680 ROISSY EN BRIE"", ""54 Ru..."


In [86]:
out.header.iloc[0]

'adresse_geographique'

In [90]:
res_rb[(res_rb['resource_id'] == 'bab27c3e-5620-47b2-8ed8-797c8192d905') & (res_rb['header'] == 'collection_isPublic')]

,index,column_type,score_ml,score_rb,header,resource_id,resource_url,sep,encoding,resource_title,dataset_title,headers,minio_url,shuffle_serie,value,check,isFoundByCsvDetective
872,31651,adresse,0.001,0.978,collection_isPublic,bab27c3e-5620-47b2-8ed8-797c8192d905,https://static.data.gouv.fr/resources/don-du-s...,",",utf-8,collections.csv,Don du sang - Lieux et horaires des collectes,"['location_address1', 'location_address2', 'lo...",http://localhost:9000/link-proxy-files/static....,"[""True"", ""True"", ""True"", ""True"", ""True"", ""True...",['booleen'],x,False
873,31660,booleen,0.001,0.982,collection_isPublic,bab27c3e-5620-47b2-8ed8-797c8192d905,https://static.data.gouv.fr/resources/don-du-s...,",",utf-8,collections.csv,Don du sang - Lieux et horaires des collectes,"['location_address1', 'location_address2', 'lo...",http://localhost:9000/link-proxy-files/static....,"[""True"", ""True"", ""True"", ""True"", ""True"", ""True...",['booleen'],x,True
874,34049,booleen,0.001,0.976,collection_isPublic,bab27c3e-5620-47b2-8ed8-797c8192d905,https://static.data.gouv.fr/resources/don-du-s...,",",utf-8,collections.csv,Don du sang - Lieux et horaires des collectes,"['location_address1', 'location_address2', 'lo...",http://localhost:9000/link-proxy-files/static....,"[""True"", ""True"", ""True"", ""True"", ""True"", ""True...",['booleen'],x,True
875,34062,adresse,0.001,0.974,collection_isPublic,bab27c3e-5620-47b2-8ed8-797c8192d905,https://static.data.gouv.fr/resources/don-du-s...,",",utf-8,collections.csv,Don du sang - Lieux et horaires des collectes,"['location_address1', 'location_address2', 'lo...",http://localhost:9000/link-proxy-files/static....,"[""True"", ""True"", ""True"", ""True"", ""True"", ""True...",['booleen'],x,False
876,39910,booleen,0.001,0.982,collection_isPublic,bab27c3e-5620-47b2-8ed8-797c8192d905,https://static.data.gouv.fr/resources/don-du-s...,",",utf-8,collections.csv,Don du sang - Lieux et horaires des collectes,"['location_address1', 'location_address2', 'lo...",http://localhost:9000/link-proxy-files/static....,"[""True"", ""True"", ""True"", ""True"", ""True"", ""True...",['booleen'],x,True
877,39904,adresse,0.001,0.972,collection_isPublic,bab27c3e-5620-47b2-8ed8-797c8192d905,https://static.data.gouv.fr/resources/don-du-s...,",",utf-8,collections.csv,Don du sang - Lieux et horaires des collectes,"['location_address1', 'location_address2', 'lo...",http://localhost:9000/link-proxy-files/static....,"[""True"", ""True"", ""True"", ""True"", ""True"", ""True...",['booleen'],x,False
878,72867,adresse,0.001,0.976,collection_isPublic,bab27c3e-5620-47b2-8ed8-797c8192d905,https://static.data.gouv.fr/resources/don-du-s...,",",utf-8,collections.csv,Don du sang - Lieux et horaires des collectes,"['location_address1', 'location_address2', 'lo...",http://localhost:9000/link-proxy-files/static....,"[""True"", ""True"", ""True"", ""True"", ""True"", ""True...",['booleen'],x,False
879,72840,booleen,0.001,0.984,collection_isPublic,bab27c3e-5620-47b2-8ed8-797c8192d905,https://static.data.gouv.fr/resources/don-du-s...,",",utf-8,collections.csv,Don du sang - Lieux et horaires des collectes,"['location_address1', 'location_address2', 'lo...",http://localhost:9000/link-proxy-files/static....,"[""True"", ""True"", ""True"", ""True"", ""True"", ""True...",['booleen'],x,True
